In [2]:
using CSV
using DataFrames
using XLSX
using Statistics

In [44]:
phylum = CSV.read("../raw-data/SCMP_Y1_phylum.csv", DataFrame)
phylum = phylum[:, Not(2:6)]

outcome = DataFrame(XLSX.readtable("../raw-data/SCMP_Y1.xlsx", "SCMP_Y1")...)
outcome = outcome[:, Not(2:10)]
outcome = outcome[:, Not(3:20)]
outcome = outcome[:,1:3]

yield_per_plant = innerjoin(phylum, outcome, on = :Link_ID)
yield_per_plant = filter(row -> !(row.Proteobacteria == "NA"), yield_per_plant)
yield_per_plant = dropmissing(yield_per_plant)
yield_per_plant = filter(row -> !(row.Yield_per_plant == "NA"), yield_per_plant)

@info("Finish cleansing the missing data...")

┌ Info: Finish cleansing the missing data...
└ @ Main In[44]:14


In [45]:
# find the median of red_little and yellow_little
sample_size = size(yield_per_plant)[1]
label_idx = size(yield_per_plant)[2]
var_idx = label_idx - 1
red_little = zeros(0)
yellow_little = zeros(0)


for i in 1:sample_size
    if yield_per_plant[i,var_idx] == "RedLittle"
        append!(red_little, yield_per_plant[i, label_idx])
    end
    if yield_per_plant[i,var_idx] == "YellowLittle"
        append!(yellow_little, yield_per_plant[i, label_idx])
    end 
end

# set the label with respect to the variety2
for i in 1:sample_size
    # threshold: 600
    if yield_per_plant[i,var_idx] == "Red" || yield_per_plant[i,var_idx] == "White" || yield_per_plant[i,var_idx] == "Yellow"
        if yield_per_plant[i,label_idx] >= 600
            yield_per_plant[i,label_idx] = 1
        else
            yield_per_plant[i,label_idx] = 0
        end
    elseif yield_per_plant[i,var_idx] == "Russet"
        if yield_per_plant[i,label_idx] >= 900
            yield_per_plant[i,label_idx] = 1
        else
            yield_per_plant[i,label_idx] = 0
        end
    elseif yield_per_plant[i,var_idx] == "RedLittle"
        if yield_per_plant[i,label_idx] >= median(red_little)
            yield_per_plant[i,label_idx] = 1
        else
            yield_per_plant[i,label_idx] = 0
        end
    else 
        if yield_per_plant[i,label_idx] >= median(yellow_little)
            yield_per_plant[i,label_idx] = 1
        else
            yield_per_plant[i,label_idx] = 0
        end
    end
end
percentage = [(i, count(==(i), yield_per_plant[:,label_idx])) for i in unique(yield_per_plant[:,label_idx])]
println("Percentage of low-yield label: ", round(percentage[1][2] / sample_size, digits = 2))

Percentage of low-yield label: 0.21


In [46]:
# Write to CSV
CSV.write("../processed-data/otu-yield-per-plant.csv", yield_per_plant)

"../processed-data/otu-yield-per-plant.csv"